# DATA TRANSFORMATION - 18_vacunacion
## 0. Previos

Se cargan las bases y las librerías a utilizar.

In [1]:
# Librerías
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import numpy as np

import generic_funcions as gf

ruta_archivos = 'C:/Users/monic/documentos/ds4a/project/datos/'

## 1. Load

In [2]:
dcc = gf.diccionario_llaves()
dcc

{'ACT': {'id': 'ID',
  'fecha': 'FE_RESULTADO',
  'fecha_no_ok': False,
  'prefi': 'act'},
 'ACT_DESAGREGADO': {'id': 'NUMERO IDENTIFICACION',
  'fecha': 'FE_RESULTADO',
  'fecha_no_ok': False,
  'prefi': 'acd'},
 'Adherencia': {'id': 'ds_identificacion',
  'fecha': 'FE_ENTREVISTA',
  'fecha_no_ok': False,
  'prefi': 'adh'},
 'Antecedentes_familiares': {'id': 'Id',
  'fecha': 'FE_ALTA',
  'fecha_no_ok': False,
  'prefi': 'ant'},
 'Antecedentes_patologicos': {'id': 'DS_IDENTIFICACION',
  'fecha': 'FE_ACTUALIZA',
  'fecha_no_ok': False,
  'prefi': 'ant'},
 'Ayudas_diagnosticas': {'id': 'Numero_Identificacion',
  'fecha': 'Fecha_Orden',
  'fecha_no_ok': False,
  'prefi': 'ayu'},
 'Biologicos Asma': {'id': 'Identificacion',
  'fecha': 'Fecha_Dcto',
  'fecha_no_ok': False,
  'prefi': 'bio'},
 'Calidad de vida relacioada en salud': {'id': 'Identificacion',
  'fecha': 'FE_ALTA',
  'fecha_no_ok': False,
  'prefi': 'cal'},
 'Disnea': {'id': 'id',
  'fecha': 'FE_ALTA',
  'fecha_no_ok': False,
  

In [3]:
modulo = 'Vacunacion'

In [4]:
base = gf.carga_datos( ruta = ruta_archivos, diccionario = dcc, modulo = modulo )
base.head()

,id,fecha,Regional_EPS_Desc,Codigo_Prestacion_Op,Descripcion_Prestacion,Codigo_Diagnostico_EPS_Op,Diagnostico_EPS_Desc,Cantidad_Autorizada,year,month,year_month
0,845676,2018-01-29,Bogota,S1120191,VACUNA PAI INFLUENZA NIÑOS/AS MAYORES DE 9 AÑOS,Z251,NECESIDAD DE INMUNIZACIÓN CONTRA LA INFLUENZA ...,1,2018,1,201801
1,686768,2018-05-20,Medellin,S1120191,VACUNA PAI INFLUENZA NIÑOS/AS MAYORES DE 9 AÑOS,Z759,PROBLEMA NO ESPECIFICADO RELACIONADO CON SERVI...,1,2018,5,201805
2,1790062,2018-06-05,Medellin,S1120191,VACUNA PAI INFLUENZA NIÑOS/AS MAYORES DE 9 AÑOS,Z251,NECESIDAD DE INMUNIZACIÓN CONTRA LA INFLUENZA ...,1,2018,6,201806
3,1142931,2018-01-10,Medellin,S1120191,VACUNA PAI INFLUENZA NIÑOS/AS MAYORES DE 9 AÑOS,Z759,PROBLEMA NO ESPECIFICADO RELACIONADO CON SERVI...,1,2018,1,201801
4,1734917,2018-06-06,Medellin,S1120191,VACUNA PAI INFLUENZA NIÑOS/AS MAYORES DE 9 AÑOS,Z251,NECESIDAD DE INMUNIZACIÓN CONTRA LA INFLUENZA ...,1,2018,6,201806


## 2. Transform

In [5]:
# pivot table to have separate variables, nans replaced with zero

ids = ['id','year','month']
base_p = base.groupby(ids)['Cantidad_Autorizada'].sum().reset_index(name='cant')

pre = dcc[modulo]['prefi'] + '_' 
base_p.columns = [ pre + s for s in base_p.columns]
base_p.rename( columns={ pre + 'id':'id', pre + 'year':'year', pre + 'month':'month' }, inplace=True)

base_p.head()

,id,year,month,vac_cant
0,500547,2018,6,1
1,500547,2019,11,1
2,537607,2018,11,1
3,537607,2019,1,1
4,539998,2018,7,1


## 3. Merge with ids table

In [6]:
ids_mensual = gf.base_ids_mensual( ruta_archivos )
base_final_vac = ids_mensual.merge( base_p, how='left')
base_final_vac.head()

,id,year,month,vac_cant
0,500547,2016,1,NaN
1,500547,2016,2,NaN
2,500547,2016,3,NaN
3,500547,2016,4,NaN
4,500547,2016,5,NaN


In [7]:
base_final_vac.shape

(53280, 4)